In [40]:
import math
def f(x, y, z):
    return (x * y + 2)**2 + (x + y)**(abs(z - 2)**3 + 2) + x * y * z

def g(x, y, z):
    return x * (y * z + 10) - math.gamma(abs(z - 3)+1) + y**abs(x) + 11 * z

def w(x, y, z):
    return (x + 7 * y)**abs(z + x) - (z + 16)**2 - 151

#적합도 함수
def F(x, y, z):
    return abs(f(x, y, z)) + abs(g(x, y, z)) + abs(w(x, y, z))


In [41]:
import copy
import random

#선택
def selection_rank_with_elite(individuals, elite_size = 0):
    sorted_individuals = sorted(individuals, key = lambda ind:ind.fitness, reverse = True)
    rank_distance = 1 / len(individuals)
    ranks = [(1 - i * rank_distance) for i in range(len(individuals))]
    ranks_sum = sum(ranks)
    selected = sorted_individuals[0:elite_size]
    for i in range(len(sorted_individuals) - elite_size):
        shave = random.random() * ranks_sum
        rank_sum = 0
    for i in range(len(sorted_individuals)):
        rank_sum += ranks[i]
        if rank_sum > shave:
            selected.append(sorted_individuals[i])
            break
    return selected

#교차
def crossover_blend(p1, p2, alpha): # blend crossover
    c1 = copy.deepcopy(p1)
    c2 = copy.deepcopy(p2)
    for i in range(len(p1)):
        l = min(c1[i], c2[i]) - alpha * abs(c2[i] - c1[i]) #p1-a(p2-p1)
        u = max(c1[i], c2[i]) + alpha * abs(c2[i] - c1[i]) #p2+a(p2-p1)
        c1[i] = round(l + random.random() * (u - l), 2)
        c2[i] = round(l + random.random() * (u - l), 2)
    return [c1, c2]

#돌연변이
def mutation_random_deviation(ind, mu, sigma, p):
    mut = copy.deepcopy(ind)
    for i in range(len(mut)):
        if random.random() < p:  #확률 계산
            mut[i] = mut[i] + random.gauss(mu, sigma)
    return mut

def constraints(x): 
    if x > 10:
        return 10
    elif x < 0:
        return 0
    return x

In [42]:
import random

class Individual:
    def __init__(self, gene_list) -> None:
        self.gene_list = [constraints(g) for g in gene_list]
        self.fitness = -F(self.gene_list[0], self.gene_list[1], self.gene_list[2])
    
    def __str__(self):
        return f'x: {self.gene_list[0]}, y: {self.gene_list[1]}, z:{self.gene_list[2]}, fitness: {self.fitness}'
    
def crossover(parent1, parent2):
    child1_genes, child2_genes = crossover_blend(parent1.gene_list,parent2.gene_list, 0.8)
    return Individual(child1_genes), Individual(child2_genes)

def mutate(ind):
    mutated_gene = mutation_random_deviation(ind.gene_list, 0, 3, 0.5)
    return Individual(mutated_gene)

def select(population):
    return selection_rank_with_elite(population, elite_size = 2)

def create_random():
    return Individual([
        round(random.uniform(-10, 10), 2),
        round(random.uniform(-10, 10), 2),
        round(random.uniform(-10, 10), 2)
        ])

def crossover_operation(population, method, prob):  #(집단, 교차방법, 확률)
    crossed_offspring = []
    to_cross = []
    result = []
    
    for ind1, ind2 in zip(population[::2], population[1::2]):
        if random.random() < prob:
            to_cross.extend([ind1, ind2])
        else:
            crossed_offspring.extend([ind1, ind2])
    
    for i in range(0, len(to_cross), 2):
        result.append(
            method, args=(to_cross[i], to_cross[i + 1])
        )
    for r in result:
        crossed_offspring.extend(r.get())
    return crossed_offspring

def mutation_operation(population, method, prob):  #(집단, 교차방법, 확률)
    mutated_offspring = []
    to_mutate = []
    result = []
    
    for ind in population:
        if random.random() < prob:
            to_mutate.append(ind)
        else:
            mutated_offspring.append(ind)
    
    for ind in to_mutate:
        result.append(method, args=(ind,))
    
    for r in result:
        mutated_offspring.append(r.get())
    return mutated_offspring

In [43]:
#Genetic algorithm parameters
POPULATION_SIZE = 400
CROSSOVER_PROBABILITY = .8
MUTATION_PROBABILITY = .4
MAX_GENERATIONS = 100

In [44]:
first_population = [create_random() for _ in range(POPULATION_SIZE)]
best_ind = random.choice(first_population)
generation_number = 0
population = first_population.copy()

while generation_number < MAX_GENERATIONS and best_ind.fitness != 0:
    generation_number += 1

    offspring = select(population)
    crossed_offspring = crossover_operation(offspring, crossover, CROSSOVER_PROBABILITY)
    mutated_offspring = mutation_operation(crossed_offspring, mutate, MUTATION_PROBABILITY)

    population = mutated_offspring.copy()
    best_of_generation = max(population, key=lambda ind: ind.fitness)

    if best_ind.fitness < best_of_generation.fitness:
        best_ind = best_of_generation

    print(f'Generation: {generation_number}, best fit: {best_ind.fitness}')
    print(f'Best Individual : {best_ind}')


OverflowError: (34, 'Result too large')

In [ ]:
print(create_random)

<function create_random at 0x0000020D18F40720>
